In [1]:
import numpy as np
import sympy as sm
from geoalg import ProjectiveGeometry3D

np.random.seed(0)

In [2]:
A = ProjectiveGeometry3D.create()
e0, e1, e2, e3 = A.blades[1]
e01, e02, e03, e12, e13, e23 = A.blades[2]
e012, e013, e023, e123 = A.blades[3]
e0123, = A.blades[4]
A.e0, A.e1, A.e2, A.e3 = e0, e1, e2, e3

In [3]:
# res = A.codegen_binary_op(lambda a, b: a * b)
# print(res)

In [4]:
# res = A.codegen_binary_op(A.sandwich)
# print(res)

In [5]:
def random_blade(grade, maxint=1):
    return sum(np.random.randint(-maxint, maxint) * b for b in A.blades[grade])

def random_multivec(maxint=1):
    return sum(random_blade(i, maxint=maxint) for i in range(len(A.blades)))

In [6]:
def test(a, b, func1, func2):
    c1 = func1(a, b)
    c2 = func2(a, b)
    if c1 != c2:
        raise RuntimeError('Products not equivalent.\na: {}\nb: {}\nc1: {}\nc2: {}'.format(a, b, c1, c2))
    assert c1 == c2

In [7]:
# Scalars are dual to pseudoscalar
# Planes are the same as vectors (vector is 0 fourth component, plane through origin)
# Points are dual to planes
# Lines are intermediate, dual to themselves
# Points with zero e0 component are ideal points (vectors / direction). Dual to planes through the origin.
# Lines through the origin have no e0*ei components, are dual to ideal lines
# Complex numbers - square root of -1

In [8]:
# Rotations are about planes not vectors
#  - think 2D case, 4D case. way more intuitive. no line out of page
# Specifying lines as point + direction is annoying
# Specifying planes as normal vector and offset separately is annoying
# Quaternion math, derivation from reflection
# Identical implementation of 2D and 3D, including pose transforms
# Treatment of translations and rotations
# Lie algebra derivation
# Autodiff with pseudoscalar

In [9]:
# lie groups = confusing
# quaternion math = confusing
# handling of infinity in projective geometry = confusing
# cross products = confusing

# a * b = |a||b| * (cos(ang) + sin(ang) * B)
# where B is a bivector of two orthogonal unit vectors in the plane of a and b
# for unit a, b
# a * b = cos(ang) + sin(ang) * B

In [10]:
# Start with geometric product idea
# Show that it is a closed ring (closed group, addition and multiplication)
# Associative but not commutative
# Reason about what different grades mean
# General product has multiple resulting grades
# Call lowest one inner product and highest one outer product
# rotors / screws
# Notion of duality
# Use in quantum mechanics

In [11]:
# Create two points
P1 = A.point(1, 0, 3)
P2 = A.point(0, 0, 3)
P1

e0*e1*e2 + 3*e0*e2*e3 + e1*e2*e3

In [12]:
# Line between two points
l = A.join(P1, P2)
l

-3*e0*e2 - e1*e2

In [13]:
# Create plane
p = A.plane(1, 1, 0, 1)
p

e0 + e1 + e2

In [14]:
p / A.product(p, p)

(e0 + e1 + e2)/2

In [15]:
# Line orthogonal to plane p through point P
A.inner(p, P1)

e0*e1 - e0*e2 - 3*e0*e3 - e1*e3 + e2*e3

In [16]:
# Plane orthogonal to plane p through line l
A.inner(p, l)

3*e0 + e1 - e2

In [17]:
A.point(0, 0, 0)

e1*e2*e3

In [18]:
A.inner(A.plane(1, 2, 3, 0), A.point(0, 0, 0))

3*e1*e2 - 2*e1*e3 + e2*e3

In [19]:
# Create a line
l = A.join(A.point(0, 0, 0), A.point(0, 0, 1))
l

e2*e3

In [20]:
# Create rotor along the line
rotor = A._rotator(l, np.deg2rad(45))
rotor

0.923879532511287 + 0.38268343236509*e2*e3

In [21]:
# Rotate point
P1

e0*e1*e2 + 3*e0*e2*e3 + e1*e2*e3

In [22]:
A.sandwich(rotor, p)

1.0*e0 + 1.0*e1 + 0.707106781186547*e2 - 0.707106781186548*e3

In [23]:
translator = A.translator(2.5, 0, 0)
translator

1 + 1.25*e0*e1

In [24]:
P = A.point(*sm.symbols('P_x P_y P_z'))
P

P_x*e0*e1*e2 + P_y*e0*e1*e3 + P_z*e0*e2*e3 + e1*e2*e3

In [25]:
t = A.translator(*sm.symbols('t_x t_y t_z'))
t

t_x*e0*e1/2 - t_y*e0*e2/2 + t_z*e0*e3/2 + 1

In [26]:
A.sandwich(t, P)

(P_x + t_z)*e0*e1*e2 + (P_y + t_y)*e0*e1*e3 + (P_z + t_x)*e0*e2*e3 + e1*e2*e3

In [27]:
def rotator(self, x, y, z):
    return self.simp(self._rotator(self.line(0, 0, 0, x, y, z), 1))
rotator(A, 0, 0, np.deg2rad(30.)).evalf()

0.877582561890373 + 0.251026625003716*e2*e3

In [28]:
axis = sm.symbols('a_x a_y a_z')
angle = sm.Symbol('theta')
rotor = A.simp(A._rotator(A.line(0, 0, 0, axis[0], axis[1], axis[2]), angle))
rotor

a_x*sin(theta/2)*e1*e2 + a_y*sin(theta/2)*e1*e3 + a_z*sin(theta/2)*e2*e3 + cos(theta/2)

In [29]:
line = A.line(0, 0, 0, axis[0], axis[1], axis[2])

In [30]:
A.normalized(line)

a_x*e1*e2/sqrt(Abs(a_x**2 + a_y**2 + a_z**2)) + a_y*e1*e3/sqrt(Abs(a_x**2 + a_y**2 + a_z**2)) + a_z*e2*e3/sqrt(Abs(a_x**2 + a_y**2 + a_z**2))

In [49]:
R = A.axis_angle(axis, angle)
R

a_x*sin(theta/2)*e1*e2 + a_y*sin(theta/2)*e1*e3 + a_z*sin(theta/2)*e2*e3 + cos(theta/2)

In [44]:
Rt = A.product(R, t)
Rt

a_x*sin(theta/2)*e1*e2 + a_y*sin(theta/2)*e1*e3 + a_z*sin(theta/2)*e2*e3 + (-a_x*t_x*sin(theta/2)/2 + a_z*t_z*sin(theta/2)/2 - t_y*cos(theta/2)/2)*e0*e2 + (-a_x*t_y*sin(theta/2)/2 + a_y*t_z*sin(theta/2)/2 + t_x*cos(theta/2)/2)*e0*e1 + (a_x*t_z*sin(theta/2)/2 + a_y*t_y*sin(theta/2)/2 + a_z*t_x*sin(theta/2)/2)*e0*e1*e2*e3 + (-a_y*t_x*sin(theta/2)/2 + a_z*t_y*sin(theta/2)/2 + t_z*cos(theta/2)/2)*e0*e3 + cos(theta/2)

In [86]:
rot = A.axis_angle([0, -1, 0], np.deg2rad(45))
rot

0.923879532511287 - 0.38268343236509*e1*e3

In [87]:
A.sandwich(rot, A.point(1, 0, 0))

0.707106781186547*e0*e1*e2 - 0.707106781186548*e0*e2*e3 + 1.0*e1*e2*e3